# Importing pandas and numpy library

In [81]:
import pandas as pd
import numpy as np

# Reading the table from wikipedia

In [95]:
x=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

# creating the DataFrame and cleaning it

In [96]:
cities=x[0]
cities.columns=cities[0][0],cities[1][0],cities[2][0]
cities=cities.iloc[1:,:]

In [97]:
cities.head()

,Postcode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [98]:
cities.replace('Not assigned',np.nan,inplace=True)
cities.dropna(inplace=True,axis=0)
cities.head()

C:\Users\stqc0\Anaconda3\lib\site-packages\pandas\core\frame.py:3798: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)
C:\Users\stqc0\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Postcode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor


In [127]:
c=cities.groupby(['Postcode','Borough'])['Neighborhood'].apply(','.join).reset_index()

In [120]:
c.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [103]:
c.shape

(102, 3)

# Getting the latitude and longitude of each place

In [112]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.columns=['Postcode','lattitude','longitutde']

In [128]:
c=pd.merge(c,geo_df,on='Postcode',how='inner')

In [253]:
c.head()

,Postcode,Borough,Neighborhood,lattitude,longitutde
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Importing nominatim from geopy to get the latitude and longitude of Toronto, Canada

In [132]:
from geopy import Nominatim
N=Nominatim()
location=N.geocode('Toronto,Canada')
lat=location.latitude
long=location.longitude

C:\Users\stqc0\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


In [133]:
print('The latitude and longitutde of Toronto are: ',lat,long)

The latitude and longitutde of Toronto are:  43.653963 -79.387207


# Importing Folium and marking all the places on the map Toronto

In [139]:
import folium 


In [184]:
Toronto=folium.Map(location=[lat,long],zoom_start=10)

for l,l1,nei in zip(c.lattitude,c.longitutde,c.Neighborhood):
    nei = folium.Popup(nei, parse_html=True)
    folium.CircleMarker(
    [l,l1],
    radius=5,
        popup=nei,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(Toronto)
    
Toronto


# Fucntion to get all the venues in Toronto

In [212]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [224]:
toron_venues = getNearbyVenues(names=c['Neighborhood'],
                                   latitudes=c['lattitude'],
                                   longitudes=c['longitutde']
                                  )



Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [220]:
print(toron_venues.shape)
toron_venues.head()

(2227, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge,Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
3,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


# Now we create a new Dataframe after one hot encoding of all the venues in the toron_venues data frame

In [256]:
venue_cat=pd.get_dummies(toron_venues[['Venue Category']])
venue_cat['Neighborhood']=toron_venues['Neighborhood']
venue_cat[['lat','long']]=toron_venues[['Venue Latitude','Venue Longitude']]
venue_cat.head()

,Venue Category_Accessories Store,Venue Category_Afghan Restaurant,Venue Category_Airport,Venue Category_Airport Food Court,Venue Category_Airport Gate,Venue Category_Airport Lounge,Venue Category_Airport Service,Venue Category_Airport Terminal,Venue Category_American Restaurant,Venue Category_Antique Shop,...,Venue Category_Vietnamese Restaurant,Venue Category_Warehouse Store,Venue Category_Wine Bar,Venue Category_Wine Shop,Venue Category_Wings Joint,Venue Category_Women's Store,Venue Category_Yoga Studio,Neighborhood,lat,long
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"Rouge,Malvern",43.807448,-79.199056
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"Rouge,Malvern",43.805630,-79.200378
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"Highland Creek,Rouge Hill,Port Union",43.785177,-79.161108
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"Highland Creek,Rouge Hill,Port Union",43.782533,-79.163085
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"Guildwood,Morningside,West Hill",43.767697,-79.189914


# Grouping the DataFrame by the neighborhood name

In [257]:
venue_cat=venue_cat.groupby('Neighborhood').mean().reset_index()

In [258]:
venue_cat.head()

,Neighborhood,Venue Category_Accessories Store,Venue Category_Afghan Restaurant,Venue Category_Airport,Venue Category_Airport Food Court,Venue Category_Airport Gate,Venue Category_Airport Lounge,Venue Category_Airport Service,Venue Category_Airport Terminal,Venue Category_American Restaurant,...,Venue Category_Video Store,Venue Category_Vietnamese Restaurant,Venue Category_Warehouse Store,Venue Category_Wine Bar,Venue Category_Wine Shop,Venue Category_Wings Joint,Venue Category_Women's Store,Venue Category_Yoga Studio,lat,long
0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,...,0.000000,0.0,0.0,0.01,0.0,0.0,0.01,0.0,43.649956,-79.383967
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.0,43.792613,-79.261140
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.0,43.815068,-79.287834
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.111111,0.0,0.0,0.00,0.0,0.0,0.00,0.0,43.741686,-79.585755
4,"Alderwood,Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.0,43.601676,-79.545335


# performing KMeans clustering on the data

In [259]:
from sklearn.cluster import KMeans



km=KMeans(n_clusters=5,random_state=1)
km.fit(venue_cat.drop(['Neighborhood','lat','long'],axis=1))

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=1, tol=0.0001, verbose=0)

In [260]:
km.labels_

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 1, 0,
       0, 0, 2, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 0, 0, 2, 0, 0, 0, 2, 0, 0,
       0, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0,
       0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2])

# New column labels added to the dataframe

In [261]:
venue_cat['label']=km.labels_

In [263]:
venue_cat.head()

,Neighborhood,Venue Category_Accessories Store,Venue Category_Afghan Restaurant,Venue Category_Airport,Venue Category_Airport Food Court,Venue Category_Airport Gate,Venue Category_Airport Lounge,Venue Category_Airport Service,Venue Category_Airport Terminal,Venue Category_American Restaurant,...,Venue Category_Vietnamese Restaurant,Venue Category_Warehouse Store,Venue Category_Wine Bar,Venue Category_Wine Shop,Venue Category_Wings Joint,Venue Category_Women's Store,Venue Category_Yoga Studio,lat,long,label
0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,...,0.0,0.0,0.01,0.0,0.0,0.01,0.0,43.649956,-79.383967,0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,43.792613,-79.261140,0
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,43.815068,-79.287834,2
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,43.741686,-79.585755,0
4,"Alderwood,Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,43.601676,-79.545335,0


# Visualizing the clusters made by the KMeans with folium

In [268]:
clustered_map=folium.Map(location=[lat,long],zoom_start=10)
col=['red','green','blue','yellow','purple']
for l,l1,nei,lab in zip(venue_cat.lat,venue_cat.long,venue_cat.Neighborhood,venue_cat['label']):
        labels1=folium.Popup(f'{nei} cluster {lab+1}',parse_html=True)
        folium.CircleMarker(
        [l,l1],
        radius=5,
        popup=labels1,
        color=col[lab],
        fill=True,
        fill_opacity=0.7,
        ).add_to(clustered_map)
clustered_map